In [1]:
import geopandas as gpd
import json
import warnings
import os
import sys

warnings.filterwarnings('ignore')


In [2]:
import math

def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi) / 2.0 * n)
    return (xtile, ytile)
  

In [3]:
filename = 'aoi.geojson'
outputFilename = 'tile-urls.json'
tileBaseUrl = "https://eratosthenes.pvretano.com/cubewerx/cubeserv/default/ogcapi/mysql_tb16/collections/waterbody_2_50k/map/Water Body 2 Style/tiles/smerc_512/%s/%s/%s.png"
zoomLevel = 8
downloadFolder = './training'

In [4]:
# Parameters
filename = "aoi.geojson"
zoomLevel = 9
tileBaseUrl = "https://eratosthenes.pvretano.com/cubewerx/cubeserv/default/ogcapi/mysql_tb16/collections/sentinel1/map/default/tiles/smerc_512/%s/%s/%s.png"
downloadFolder = "./training2"


In [5]:
gj = gpd.read_file(filename)
gj.head()

,geometry
0,"POLYGON ((-111.40137 62.99516, -107.27051 62.9..."


In [6]:
minLat = gj.head().bounds.miny[0]
maxLat = gj.head().bounds.maxy[0]
minLon = gj.head().bounds.minx[0]
maxLon = gj.head().bounds.maxx[0]
maxLat

64.69910544204765

In [7]:
topLeftTile = deg2num(minLat,minLon,zoomLevel)
topLeftTile

(97, 139)

In [8]:
bottomRightTile = deg2num(maxLat,maxLon,zoomLevel)
bottomRightTile

(103, 134)

In [9]:
resultUrls = {}

result = ""
for x in range(topLeftTile[0], bottomRightTile[0] + 1):
    for y in range(bottomRightTile[1], topLeftTile[1] + 1):
        result = tileBaseUrl%(zoomLevel, y, x)
        resultUrls["%s_%s_%s.png"%(zoomLevel, y, x)] = result
        
resultUrls


{'9_134_97.png': 'https://eratosthenes.pvretano.com/cubewerx/cubeserv/default/ogcapi/mysql_tb16/collections/sentinel1/map/default/tiles/smerc_512/9/134/97.png',
 '9_135_97.png': 'https://eratosthenes.pvretano.com/cubewerx/cubeserv/default/ogcapi/mysql_tb16/collections/sentinel1/map/default/tiles/smerc_512/9/135/97.png',
 '9_136_97.png': 'https://eratosthenes.pvretano.com/cubewerx/cubeserv/default/ogcapi/mysql_tb16/collections/sentinel1/map/default/tiles/smerc_512/9/136/97.png',
 '9_137_97.png': 'https://eratosthenes.pvretano.com/cubewerx/cubeserv/default/ogcapi/mysql_tb16/collections/sentinel1/map/default/tiles/smerc_512/9/137/97.png',
 '9_138_97.png': 'https://eratosthenes.pvretano.com/cubewerx/cubeserv/default/ogcapi/mysql_tb16/collections/sentinel1/map/default/tiles/smerc_512/9/138/97.png',
 '9_139_97.png': 'https://eratosthenes.pvretano.com/cubewerx/cubeserv/default/ogcapi/mysql_tb16/collections/sentinel1/map/default/tiles/smerc_512/9/139/97.png',
 '9_134_98.png': 'https://eratosth

In [10]:
import json

with open(outputFilename, 'w') as outfile:
    json.dump(resultUrls, outfile)

In [11]:
import shutil

# clear the target folder
if os.path.exists(downloadFolder) and os.path.isdir(downloadFolder):
    shutil.rmtree(downloadFolder)

# re-create the target folder
if not os.path.exists(downloadFolder):
    os.makedirs(downloadFolder)

'start to download %s tiles'%(len(resultUrls))


'start to download 42 tiles'

In [12]:
import requests
    
for fileName, targetUrl in resultUrls.items():
    r = requests.get(targetUrl, allow_redirects=True)
    open(downloadFolder + '/' + fileName, 'wb').write(r.content)
    
'downloaded %s tiles to %s'%(len(resultUrls), downloadFolder)

'downloaded 42 tiles to ./training2'